In [117]:
import pandas as pd
import numpy as np
import random
import math

# Consolidating labeling up to 12-31 to ponder next labeling move
- Combine all four manual-label iterations until now with auto labeling
- incorporate info about whether a channel is False for ALL topics
- Find number of channels that still need to be labeled for P1/P2/P3


## Combine all other labeling iterations to one dictionary
- I am in the process of "updating" my labels with new codebook, but we work with what we have right now
- Four iterations of manual labeling until now, plus automatic labeling from channel lists
- Can use "apply" rather than iterate through df next time
- Stopped the labeled_dict_extra code because I got lazy (it was in the effort of creating a new dataset with all current manual labels)
- ^However, I should continue this when I pick up again

In [118]:
# dict of {channel -> {topic --> True/False (on-topic?)} }
labeled_dict = {}
labeled_dict_extra = {}
topics = ['alt-right', 'antitheist', 'politics-left', 'politics-right']

In [119]:
auto_channels_path = '../../../burst/communities/'

In [120]:
channel_paths_dict = {'alt-right': auto_channels_path+'alt-right/alt-right_channels.csv',
                     'antitheist': auto_channels_path+'antitheist/antitheist_channels.csv',
                     'politics-left': auto_channels_path+'politics/channels_left.csv',
                     'politics-right': auto_channels_path+'politics/channels_right.csv',
                     }

for topic in channel_paths_dict.keys():
    channels_filepath = channel_paths_dict[topic]
    channels_df = pd.read_csv(channels_filepath, header=None)
    
    for channel_id in channels_df[0]:
        if channel_id not in labeled_dict.keys():
            labeled_dict[channel_id] = {}
            labeled_dict_extra[channel_id] = {}

        labeled_dict[channel_id][topic] = True
        
        labeled_dict_extra[channel_id][topic] = {'labeling_setting':'automatic','recheck':False}
        
        for other_topic in topics:
            if other_topic != topic and other_topic not in labeled_dict[channel_id].keys():
                labeled_dict[channel_id][other_topic] = False
                labeled_dict_extra[channel_id][topic] = {'labeling_setting':'automatic','recheck':False}


In [121]:
len(labeled_dict.keys())

650

In [122]:
len(labeled_dict.keys())

650

In [123]:
# Stain topic validation
df_stv = pd.read_csv('./labeled_12-31/channel labeling - stain_topic_validation.csv')
df_stv.head()

# easier logic than for scrub strategy characterization spreadsheet
# columns are topic, channel_id and on-topic
for index, row in df_stv.iterrows():
    
    if 'topic' in row.keys() and 'channel_id' in row.keys():
        topic = row['topic']
        channel_id = row['channel_id']
        
        if topic in topics and channel_id[:2] == 'UC':
            on_topic = row['on-topic?']

            if channel_id not in labeled_dict.keys():
                labeled_dict[channel_id] = {}
                labeled_dict_extra[channel_id] = {}
                
            if on_topic == 'no':
                labeled_dict[channel_id][topic] = False
            elif on_topic == 'yes':
                labeled_dict[channel_id][topic] = True
            
            if on_topic == 'no' or on_topic == 'yes':
                if type(row['12/30- check again'])==str and len(row['12/30- check again'])>0:
                    labeled_dict_extra[channel_id][topic] = {'labeling_setting':'stain topic val','recheck':True}
                else:
                    labeled_dict_extra[channel_id][topic] = {'labeling_setting':'stain topic val','recheck':False}

            
            # Clearly not of any other topic
            if type(row['Potential other category']) != str:
                for other_topic in topics:
                    if other_topic != topic and other_topic not in labeled_dict[channel_id].keys():
                        labeled_dict[channel_id][other_topic] = False
                        labeled_dict_extra[channel_id][topic] = {'labeling_setting':'stain topic val','recheck':False}


In [124]:
len(labeled_dict.keys())

800

In [125]:
# scrub end understimation
df_scrub_end = pd.read_csv('./labeled_12-31/channel labeling - end_scrub_underestimation.csv')
df_scrub_end.head()

,Unnamed: 0,topic,strategy,note,rank,channel_id,stain,should_be_stain,potential other category,12/30- check again,notes,shuffled_idx
0,2555,alt-right,delete,0,0,UCpar38sKgMVxiaO80LZFdwA,False,False,politics,NaN,pagan tradition commenter on modern news,46
1,2556,alt-right,delete,0,1,UC4YUKOBld2PoOLzk0YZ80lw,False,False,NaN,NaN,minecrafter,27
2,2557,alt-right,delete,0,2,UCiIWdzEVNH8okhlapR9a-xA,False,False,NaN,NaN,NaN,24
3,2558,alt-right,delete,0,3,NaN,False,False,NaN,NaN,NaN,18
4,2559,alt-right,delete,0,4,UCX6OQ3DkcsbYNE6H8uQQuVA,False,False,NaN,NaN,NaN,48


In [126]:
def add_to_labeled_dict(x, labeled_dict, labeled_dict_extra):
    #print(x)
    topic = x['topic']
    channel_id = x['channel_id']
    should_be_stain = x['should_be_stain']
    pot_other_cat = x['potential other category']

    if channel_id not in labeled_dict.keys():
        labeled_dict[channel_id] = {}
        #labeled_dict_extra[channel_id] = {}
    
    if topic in topics and type(channel_id) == str and channel_id[:2] == 'UC':
        labeled_dict[channel_id][topic] = should_be_stain
        
        #if type(x['12/30- check again'])==str and len(x['12/30- check again'])>0:
        #    labeled_dict_extra[channel_id][topic] = {'labeling_setting':'scrub end under','recheck':True}
        #else:
        #    labeled_dict_extra[channel_id][topic] = {'labeling_setting':'scrub end under','recheck':False}

    if type(pot_other_cat) != str:
        for other_topic in topics:
            if other_topic != topic and other_topic not in labeled_dict[channel_id].keys():
                labeled_dict[channel_id][other_topic] = False
                #labeled_dict_extra[channel_id][topic] = {'labeling_setting':'scrub end under','recheck':False}

In [127]:
df_scrub_end[['topic','channel_id','should_be_stain','potential other category','12/30- check again']].apply(lambda x:
     add_to_labeled_dict(x, labeled_dict, labeled_dict_extra), axis=1
)

0       None
1       None
2       None
3       None
4       None
        ... 
1395    None
1396    None
1397    None
1398    None
1399    None
Length: 1400, dtype: object

In [128]:
len(labeled_dict.keys())

1387

In [129]:
df_12_26 = pd.read_csv('./labeled_12-31/Manual labeling 12_26 - homepage_to_label_12-26.csv')
df_12_26.head()

,rank,video_id,channel_id,topic,strategy,should_be_stain,potential other category,12/31 recheck?,comments
0,5,jpmLEdCeevo,UC81DuEHqyRUGhj6rMcEP6cw,alt-right,delete,False,NaN,NaN,Japanese
1,6,Dx5qFachd3A,UCJhjE7wbdYAae1G25m0tHAA,alt-right,delete,False,NaN,NaN,NaN
2,14,PLspFX498zc,UCYac9irNGo3pSrvc_L8HERw,alt-right,delete,NaN,NaN,NaN,NaN
3,5,YGsG2aCtt8E,UCJI4ZOQ_0R1HbWmIPY3DTLg,alt-right,delete,False,NaN,NaN,emergency responses
4,13,ZwqzcGVVJIc,UCCXoCcu9Rp7NPbTzIvogpZg,alt-right,delete,NaN,NaN,NaN,NaN


In [130]:
df_12_26[['topic','channel_id','should_be_stain','potential other category']].apply(lambda x:
     add_to_labeled_dict(x, labeled_dict, labeled_dict_extra), axis=1
)

0      None
1      None
2      None
3      None
4      None
       ... 
385    None
386    None
387    None
388    None
389    None
Length: 390, dtype: object

In [131]:
len(labeled_dict.keys())

1669

## Apply these labels to our P1/2/3/ data

In [132]:
df = pd.read_csv('../all_cleaned_labeled.csv')
df = df[(df['topic']!='random')&(df['rank']<10)]
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,video_id,channel_id,rank,component,watch_video_id,bot_name,phase,phase_level,homepage_level,videopage_level,time,topic,strategy,note,rerun,stain,channel_type
0,0,0,0,PEWFu0aZroQ,UCXIJgqnII2ZOINSWNOGFThA,0,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,2022-08-07 04:54:08.682177,alt-right,delete,0,0,False,off-list
1,1,1,1,XmiG4KzZ4sg,UCPKAKrjoMz7POptCloy7AIQ,1,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,2022-08-07 04:54:08.682192,alt-right,delete,0,0,False,off-list
2,2,2,2,Lh0b00vL1nI,UCmRZTrJYqOMzP5J0r0snxYg,2,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,2022-08-07 04:54:08.682195,alt-right,delete,0,0,False,off-list
3,3,3,3,k6ucIShHW7Q,UCJquYOG5EL82sKTfH9aMA9Q,3,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,2022-08-07 04:54:08.682199,alt-right,delete,0,0,False,off-list
4,4,4,4,0GCDhADokLY,UC2ggrxKhqgTRx7We9vjQRCQ,4,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,2022-08-07 04:54:08.682202,alt-right,delete,0,0,False,off-list


In [133]:
df_homepage = df[
    (df['component']=='homepage')&
    (df['homepage_level'].isin([0,39,79]))
]

In [134]:
df_videopage = df[(df['phase']=='videopage_experiment')]
df_videopage.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,video_id,channel_id,rank,component,watch_video_id,bot_name,phase,phase_level,homepage_level,videopage_level,time,topic,strategy,note,rerun,stain,channel_type
0,0,0,0,PEWFu0aZroQ,UCXIJgqnII2ZOINSWNOGFThA,0,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,2022-08-07 04:54:08.682177,alt-right,delete,0,0,False,off-list
1,1,1,1,XmiG4KzZ4sg,UCPKAKrjoMz7POptCloy7AIQ,1,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,2022-08-07 04:54:08.682192,alt-right,delete,0,0,False,off-list
2,2,2,2,Lh0b00vL1nI,UCmRZTrJYqOMzP5J0r0snxYg,2,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,2022-08-07 04:54:08.682195,alt-right,delete,0,0,False,off-list
3,3,3,3,k6ucIShHW7Q,UCJquYOG5EL82sKTfH9aMA9Q,3,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,2022-08-07 04:54:08.682199,alt-right,delete,0,0,False,off-list
4,4,4,4,0GCDhADokLY,UC2ggrxKhqgTRx7We9vjQRCQ,4,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,2022-08-07 04:54:08.682202,alt-right,delete,0,0,False,off-list


In [135]:
videopage_experiment_levels = []

vp_exp_level = 0
vp_level = 0
bot_name = None

for index, row in df_videopage.iterrows():
    if row['bot_name'] != bot_name:
        bot_name = row['bot_name']
        vp_exp_level = 0
        vp_level = 0
    if row['videopage_level'] != vp_level:
        vp_level = row['videopage_level']
        vp_exp_level += 1
    
    videopage_experiment_levels.append(vp_exp_level)

In [136]:
df_videopage['exp_level'] = videopage_experiment_levels

<ipython-input-136-e031aa8372a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_videopage['exp_level'] = videopage_experiment_levels


In [137]:
df_videopage.groupby(['bot_name'])['exp_level'].apply(lambda x: x.unique())

bot_name
alt-right_delete_0        [0, 1, 2]
alt-right_delete_1        [0, 1, 2]
alt-right_delete_2        [0, 1, 2]
alt-right_delete_3        [0, 1, 2]
alt-right_delete_4        [0, 1, 2]
                            ...    
politics-right_watch_0    [0, 1, 2]
politics-right_watch_1    [0, 1, 2]
politics-right_watch_2    [0, 1, 2]
politics-right_watch_3    [0, 1, 2]
politics-right_watch_4    [0, 1, 2]
Name: exp_level, Length: 139, dtype: object

In [138]:
df_homepage['stain_combined'] = df_homepage[['channel_id','topic']].apply(
    lambda x: labeled_dict[x.channel_id][x.topic] if x.channel_id in labeled_dict.keys() and x.topic in labeled_dict[x.channel_id].keys() else 'todo',
    axis=1
)

<ipython-input-138-b8d460e0c855>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_homepage['stain_combined'] = df_homepage[['channel_id','topic']].apply(


In [139]:
len(df_homepage[
    ( (df_homepage['homepage_level']==39) | (df_homepage['homepage_level']==0) )&
    (df_homepage['stain_combined']=='todo')
]['channel_id'].unique())

523

In [140]:
len(df_homepage[
    ( (df_homepage['homepage_level']==39) )&
    (df_homepage['stain_combined']=='todo')
]['channel_id'].unique())

225

### This is the last dataset to finish labeling P2/P3 for homepage
- Caveat: Still need to do rechecks and update codebook

In [142]:
df_homepage.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'video_id',
       'channel_id', 'rank', 'component', 'watch_video_id', 'bot_name',
       'phase', 'phase_level', 'homepage_level', 'videopage_level', 'time',
       'topic', 'strategy', 'note', 'rerun', 'stain', 'channel_type',
       'stain_combined'],
      dtype='object')

In [145]:
df_homepage[
    ( (df_homepage['homepage_level']==39) | (df_homepage['homepage_level']==79) )&
    (df_homepage['stain_combined']=='todo')
][['rank','video_id','channel_id','topic','strategy']].to_csv('./homepage_manual_labeling_1-2.csv',index=False)
